<img src="https://cdn4.iconfinder.com/data/icons/redis-2/1451/Untitled-2-512.png" alt="" width="200"/>

# Redis, le DBM du futur ?

- Antonio Giménez Nadal
- Pierre Sammut
- Cyril Scoul
- Antoine Stevan

[Toni] Redis est un gestionnaire de base de données de type NoSQL très populaire au près des grandes entreprises. 

# Redis, dans le monde des DBMs


[Toni] Avant d'entrer dans le vif du sujet, contextualisons où se trouve Redis parmi les différents DBMS.

## SQL vs NoSQL


[Toni] Quelle est la différence entre les langages SQL et NoSQL ?

<img src="https://th.bing.com/th/id/OIP.W6F1WcYqgaogrQV99OvBVgHaD9?pid=ImgDet&rs=1" alt="" width="600"/>


[Toni] Dans la famille SQL on trouve deux type the gesteurs. Les relationnels et les analytics, exploitables par différents logiciels tel que postgrestSQL et d'autres différentes versions commerciales.

Dans la famille des bases de données NoSQL, on peut distinguer 4 types familles :

   - Wide-colum (Exploitation par Cassandra par exemple) ;
   - Graphs (Neo4j) ;
   - Documents (MongoDB) ;
   - Key-Value, (Redis).

Redis est un gestionnaire de base de données NoSQL, en particulier pour les données sous la forme Key-Value. 

Néanmoins, moyennement l'installation de plugins, Redis peut gérer tous type de base de données NoSQL.

# Redis, comment ça marche ? 

Le nom Redis vient de **Re**mote **Di**ctionary **S**erver. 

Il propose :

   - Une **base de données en mémoire** : ces bases de données permettent au SGBD d’enregistrer toutes les données directement dans la mémoire vive. Ceci permet des délais d’accès très courts – même en cas de grands volumes de données non structurées.  
   
   - Un **système clé-valeur**: chaque entrée de la base de données se voit attribuer une clé. Elle permet de consulter les données ultérieurement en toute simplicité. Les entrées ne sont donc pas liées les unes aux autres et ne doivent donc pas être interrogées dans plusieurs tableaux. Les informations sont directement disponibles.

Le système peut supporter les types de données suivants : 

   - **Strings** : une suite de caractères d’une taille maximale de 512 Mo 
   - **Hashes** : une entrée avec plusieurs champs 
   - **Lists** : un ensemble de strings triés par ordre de saisie 
   - **Sets** : un ensemble de strings non triés 
   - **Sorted sets** : un ensemble de strings triés par l’utilisateur 
   - **Bitmaps** : un ensemble d’opérations au niveau des bits 
   - **HyperLogLogs** : une estimation basée sur des valeurs précises 
   - **Streams** : une liste de strings ou de paires clé-valeur complexes

# Redis vs PostgreSQL

Nous allons analyser les différences à l'aide d'un cas pratique que nous avons fait.

## Base de donnes

Nous avons retenu une base de données extraite du jeu Animal Crossing pour réaliser notre étude.

La base de données contient 30 csv, chacun répertoriant divers objets, villageois, vêtements et autres objets de collection du jeu. Les données ont été collectées par un groupe de fans d'Animal Crossing qui continue de collaborer et de créer ces feuilles de calcul pour un usage public. La base de données contient les données originales et la liste complète des contributeurs et des données brutes. Au moment de la rédaction, la seule différence entre la feuille de calcul et cette version est que la version Kaggle omet toutes les colonnes avec des images des éléments, mais est par ailleurs identique.

Source : https://www.kaggle.com/jessicali9530/animal-crossing-new-horizons-nookplaza-dataset?fbclid=IwAR132yW030ZcxYDtzeSpl9Zp7m-7_ZOUUw20d3P0tdFwuOE3PXYZ9J7TDhs


In [9]:
Montrer la forme des tables avec **seecsv ? Insérer le code.  Comment il marhce c'est code parce que c'est bash **

SyntaxError: invalid syntax (<ipython-input-9-d87383325fb0>, line 1)

## Pré-traitement nécessaire pour pouvoir exploiter la base de données avec Redis                                          

Explication : scripts/imports/redis_import_main.py tables.

In [ ]:
# %load scripts/imports/redis_import_main.py
import argparse
import os
import traceback
import redis
import csv
import glob

def main(db_path, script=False, auto=False, verbose=False, clean=False):
    # gets the name of the directory where the script is, to allow
    # executing the script from anywhere.
    directory = os.path.dirname(os.path.realpath(__file__))

    #DATABASE IMPORT TO REDIS
    csv_files = [f for f in glob.glob(os.path.join(db_path, "*.csv"))]
    csv_files.sort()
    elem_id = 0
    output_filename = os.path.join(directory, "import_db.redis")

    if clean:
        if os.path.exists(output_filename):
            print(f"removing {output_filename}")
            os.remove(output_filename)
        else:
            print("no .redis file to remove... Skipping.")

    if not script and not auto:
        print("nothing to do... Aborting.")
        return
    if auto: #Open Redis DBM
        r = redis.Redis() #host='localhost', port=6379, db=0, password=None, socket_timeout=None
    if script:
        redis_script = open(output_filename, "w")

In [ ]:
    for csv_file in csv_files:
            try:
                csvfile = open(csv_file, newline='')
                csvreader = csv.reader(csvfile, delimiter=',')
                first_row = True
                for row in csvreader:
                    if first_row == True:
                        headers = row
                        headers[0] = headers[0][1:]
                        first_row = False
                    else: #Add to Redis                
                        if auto: #Import directly 
                            line = {f"{'_'.join(header.split(' '))}" : value.replace('\n', ';') for header, value in list(zip(headers, row))}
                            r.hmset(f'{elem_id}', line)
                            if verbose:
                                print(r.hgetall(f'{elem_id}'))
                        if script: #Create redis script: ex: HMSET elem_id field1 "Hello" field2 "World"
                            line = " ".join(f"{'_'.join(header.split(' '))} " + value.replace('\n', ';') for header, value in list(zip(headers, row)) )
                            redis_line = f'HMSET {elem_id} {line}\n'
                            redis_script.write(redis_line)
                            if verbose:
                                print(redis_line)
                        elem_id +=1 #at the end so we keep python index notation
                csvfile.close()
            except:
                print(traceback.format_exc())
    if script:
        redis_script.close()

In [ ]:
if __name__ == "__main__":
    # create a parser to change behaviour from outside.
    parser = argparse.ArgumentParser()
    parser.add_argument("tables", type=str, nargs=1,
            help="the location of the tables, from the execution location (is required).")
    parser.add_argument('-s', "--script", default=False, action="store_true",
            help="if raised, the execution will generate a redis import script (defaults to False).")
    parser.add_argument('-a', "--auto", default=False, action="store_true",
            help="allows the script to automatically load the database inside a redis server. make sure a redis server is activated before hand. (defaults to False).")
    parser.add_argument('-v', "--verbose", default=False, action="store_true",
            help="turns on verbose (defaults to False).")
    parser.add_argument('-c', "--clean", default=False, action="store_true",
            help="cleans the workspace from generated .redis file (defaults to False).")

    args = parser.parse_args()

    #EXECUTION PARAMETERS
    script = args.script
    auto = args.auto
    verbose = args.verbose
    db_path = args.tables[0]
    clean = args.clean

    # the actual redis database generation.
    main(db_path, script=script, auto=auto, verbose=verbose, clean=clean)

Les données se présentent alors sous la forme clef-valeur : 
 
   - La clef est un entier compris entre 0 et 16144, 16 144 correspondant à l'addition du nombre de lignes total de toutes les relations contenues dans la base de données Animal Crossing.
   - La valeur associée (qui est une ligne) se présente sous la forme :  

          1) "Nom de la colonne1"/"Donnée associée" 
          2) "Nom de la colonne2"/"Donnée associée" 
          ...
          N) "Nom de la colonneN"/"Donnée associée"

Les requêtes sur PostgreSQl étant incompatibles avec les reqûetes Redis, on choisit de contourner le problème en passant par le langage Python afin de pouvoir les traduire.

## Quelques requêtes SQL sur PostgreSQL et leurs équivalent en Redis en passant par python

Nous avons fait plus de 30 requêtes. Vous pouvez les consulter sur le github du projet. Cependant, nous allons maintenant en discuter quelques-uns juste pour voir les différences.

Commenter la forme des reqûetes en direct.

### Query 1

In [ ]:
# %load scripts/queries/psql/query_01/query.psql
SELECT AVG(sell)
FROM accessories;

In [ ]:
# %load scripts/queries/redis/query_01/query.py
import redis

r = redis.Redis()

#keys corresponding to tables
accessories = range(0,222)
#Query 2
list_sell = []
for key in accessories :
    elem = r.hmget(key,'Sell')
    list_sell.append(int(elem[0].decode()))
results2 = sum(list_sell)/len(list_sell)
print("Query 2 :", results2)

### Query 7

In [ ]:
# %load scripts/queries/psql/query_07/query.psql
SELECT name, category, buy, sell, catalog, artist, museum_description
FROM art
WHERE size_of='1x1'
AND hha_concept_1='expensive'
AND hha_concept_2='folk art'
AND tag='Sculpture';

In [ ]:
# %load scripts/queries/redis/query_07/query.py
import redis

r = redis.Redis()

#keys corresponding to tables
art = range(306,376)

#Query 8
results8 = []
for key in art :
    elem_decoded = r.hmget(key,'Size')[0].decode()
    elem_decoded2 = r.hmget(key,'HHA_Concept_1')[0].decode()
    elem_decoded3 = r.hmget(key,'HHA_Concept_2')[0].decode()
    elem_decoded4 = r.hmget(key,'Tag')[0].decode()
    if (elem_decoded == '1x1') and (elem_decoded2 == 'expensive') and (elem_decoded3 == 'folk art') and (elem_decoded4 == 'Sculpture') :
        result8 = []
        result8.append(r.hmget(key,'Name')[0].decode())
        result8.append(r.hmget(key,'Category')[0].decode())
        result8.append(r.hmget(key,'Buy')[0].decode())
        result8.append(r.hmget(key,'Sell')[0].decode())
        result8.append(r.hmget(key,'Catalog')[0].decode())
        result8.append(r.hmget(key,'Artist')[0].decode())
        result8.append(r.hmget(key,'Museum_Description')[0].decode())
        results8.append(result8)

print("Query 8 :", results8)

### Query 18

In [ ]:
# %load scripts/queries/psql/query_18/query.psql
SELECT name, buy, sell, catalog, color_1, color_2, source, outdoor FROM housewares WHERE hha_concept_1='expensive' AND tag='Toy';

In [ ]:
# %load scripts/queries/redis/query_18/query.py
import redis

r = redis.Redis()

#keys corresponding to tables
housewares = range(3393,6668)

#Query 19
results19 = []
for key in housewares:
    elem_decoded = r.hmget(key,'HHA_Concept_1')[0].decode()
    elem_decoded1 = r.hmget(key,'Tag')[0].decode()
    if (elem_decoded == 'expensive') and (elem_decoded1 == 'Toy'):
        result19 = []
        result19.append(r.hmget(key,'Name')[0].decode())
        result19.append(r.hmget(key,'Buy')[0].decode())
        result19.append(r.hmget(key,'Sell')[0].decode())
        result19.append(r.hmget(key,'Catalog')[0].decode())
        result19.append(r.hmget(key,'Color_1')[0].decode())
        result19.append(r.hmget(key,'Color_2')[0].decode())
        result19.append(r.hmget(key,'Source')[0].decode())
        result19.append(r.hmget(key,'Outdoor')[0].decode())
        results19.append(result19)
print("Query 19 :", results19)

### Query 34

In [ ]:
# %load scripts/queries/psql/query_34/query.psql
SELECT name, personality, hobby, birthday, catchphrase, favorite_song, style_1, style_2, color_1, color_2, wallpaper, furniture_list FROM villagers WHERE species='Bird' AND gender='Male' AND flooring='tatami';

In [ ]:
# %load scripts/queries/redis/query_34/query.py
import timeit
import redis

r = redis.Redis()
villagers = range(15049,15440) #Keys corresponding to tables
Results34 = []
for key in villagers:
    elem_decoded1 = r.hmget(key,'Species')[0].decode()
    elem_decoded2 = r.hmget(key,'Gender')[0].decode()
    elem_decoded3 = r.hmget(key,'Flooring')[0].decode()

    if (elem_decoded1 == 'Bird') and (elem_decoded2 == 'Male') and (elem_decoded3 == 'tatami'): 
        Result34 = []
        Result34.append(r.hmget(key,'Name')[0].decode())
        Result34.append(r.hmget(key,'Personality')[0].decode())
        Result34.append(r.hmget(key,'Hobby')[0].decode())
        Result34.append(r.hmget(key,'Birthday')[0].decode())
        Result34.append(r.hmget(key,'Catchphrase')[0].decode())
        Result34.append(r.hmget(key,'Favorite_Song')[0].decode())
        Result34.append(r.hmget(key,'Style_1')[0].decode())
        Result34.append(r.hmget(key,'Style_2')[0].decode())
        Result34.append(r.hmget(key,'Color_1')[0].decode())
        Result34.append(r.hmget(key,'Color_2')[0].decode())
        Result34.append(r.hmget(key,'Wallpaper')[0].decode())
        Result34.append(r.hmget(key,'Furniture_List')[0].decode())
        Results34.append(Result34)
#print('Query 34 : ', Results34)
timeit.timeit()

## Temps d'execution des requêtes avec PostgreSQL et Redis


Discuter la différence des temps d'execution.

### PostgrestSQL


|command | time [ms] (not cached) | time [ms] (cached) | error [ms] |
|:- | :- | :- | :- |
|INSERT INTO ... (...) VALUES (...);            |   ~21.5    | ~1.5   |        ~0.8      |
|DELETE FROM ... WHERE ...;                     |   ~21.5    | ~1.5   |        ~0.6      |
|UPDATE ... SET ... WHERE ...;                  |   ~22      | ~1.5   |        ~0.8      |
|ALTER TABLE ... ADD ... ...;                   |   ~22      | ~1.8   |        ~0.7      |
|ALTER TABLE ... DROP COLUMN ...;               |   ~22      | ~1.8   |        ~0.7      |
|ALTER TABLE ... ALTER COLUMN ...; (same type)  |   ~22      | ~1.5   |        ----      |
|ALTER TABLE ... ALTER COLUMN ...; (other type) |   ~48      | ~35    |        ----      |
|ALTER TABLE ... RENAME COLUMN ...;             |   ~30      | ~1.5   |        ~0.8      |


### PostgrestSQL

times over 101 trials

| query | time: mean  | time: std  | rank |
|:-  | :- | :- | :- |
|0  |  1,45243 | 0,00254431    |  19  | 
|1  |  1,44999 | 0,000198012   |  8   |  
|2  |  1,57168 | 0,0129673     |  36  |  
|3  |  1,75741 | 4,07965e-05   |  35  |  
|4  |  2,17965 | 0,000363189   |  18  | 
|5  |  1,67916 | 0,00608565    |  24  |  
|6  |  1,71129 | 0,0093822     |  21  |  
|7  |  1,81377 | 7,66179e-05   |  4   | 
|8  |  7,20971 | 0,023454      |  32  |  
|9  |  1,47102 | 0,00229057    |  14  |  
|10 |  0,03511 | 0,0494328     |  12  |  
|11 |  1,66974 | 0,00355627    |  17  |  
|12 |  1,9723  | 0,00401       |  20  |  
|13 |  1,29768 | 0,00580305    |  34  |  
|14 |  1,99219 | 0,00230749    |  30  |  
|15 |  1,79065 | 0,0568515     |  38  |  


| query | time: mean  | time: std  | rank |
|:-  | :- | :- | :- |
|16 |  1,68871 | 0,0014637     |  29  | 
|17 |  1,96511 | 0,001404      |  31  | 
|18 |  2,61801 | 0,000498514   |  7   |  
|19 | 10,3022  | 0,0108658     |  37  |  
|20 |  1,96095 | 0,000293536   |  15  | 
|21 |  2,32944 | 0,000441797   |  25  |  
|22 |  1,39298 | 0,00109255    |  28  |  
|23 |  1,55501 | 9,85087e-05   |  3   | 
|24 |  2,35178 | 0,00375925    |  33  |  
|25 |  1,78952 | 0,00820707    |  6   |  
|26 |  1,62799 | 9,95037e-07   |  16  |  
|27 |  1,60054 | 0,00711849    |  5   |  
|28 |  1,78355 | 0,006015      |  11  |  
|29 |  1,89648 | 0,00462493    |  26  |  
|30 |  1,9006  | 0,00471648    |  27  |  
|31 |  1,83267 | 0,0132011     |  2   |  
|32 |  2,01368 | 0,00484384    |  23  |  
|33 |  1,74344 | 0,00163584    |  9   |  
|34 |  1,92456 | 0,00622495    |  0   |  
|35 |  3,58048 | 0,000147266   |  1   |  
|36 |  3,94201 | 0,0145285     |  22  |  
|37 |  1,79605 | 4,97519e-06   |  13  |  
|38 |  1,89671 | 0,0041085     |  10  |

### Redis

|rank | query | time execution [ms] |
|:- | :- | :- |
| 1 | query_37 | 0.01439669999990656 |
| 2 | query_28| 0.014632300000130272|
| 3 | query_19| 0.01521679999996195.|
| 4 | query_13| 0.015262000000006992.|
| 5 | query_33| 0.015263000000004467.|
| 6 | query_26| 0.015269199999920602.|
| 7 | query_27| 0.015377100000023347.|
| 8 | query_24| 0.015502299999980096.|
| 9 | query_15| 0.015539600000010978.|
| 10 | query_09|  0.01559769999994387.|
| 11 | query_02|  0.015759400000007417.|
| 12 | query_01|  0.015943000000007146.|
| 13 | query_06|  0.01599900000002208.|
| 14 | query_11|  0.0160500999999158.|
| 15 | query_03|  0.016089100000016288.|
| 16 | query_07|  0.016349499999989803.|

|rank | query | time execution [ms] |
|:- | :- | :- |
| 17 | query_29|  0.016535800000156087.|
| 18 | query_00|  0.016543699999999717.|
| 19 | query_25|  0.01668029999996179.|
| 20 | query_34|  0.017864599999938946.|
| 21 | query_21|  0.01807379999991099.|
| 22 | query_16|  0.018181600000048093.|
| 23 | query_22|  0.019896799999969517.|
| 24 | query_35|  0.02064180000002125.|
| 25 | query_17|  0.020691200000101162.|
| 26 | query_10|  0.021966999999904147.|
| 27 | query_30|  0.02466950000007273.|
| 28 | query_31|  0.025483399999984613.|
| 29 | query_05|  0.025587099999995644.|
| 30 | query_12|  0.0265006000000767.|
| 31 | query_23|  0.02941740000005666.|
| 32 | query_38|  0.02982719999999972.|
| 33 | query_20|  0.02997779999998329.|
| 34 | query_04|  0.030045499999999947.|
| 35 | query_32|  0.03298699999982091.|
| 36 | query_18|  0.04078839999999673.|
| 37 | query_14|  0.048689399999943817.|
| 38 | query_36|  too long|
| 39 | query_08|  too long|

# Redis, quand l'utiliser ?  Conclusions !

Discuter l'efficacité en fonction de la nature des données, temps d'execution.
Quelles entreprises utilises déjà Redis ?

- Avantage en termes de rapidité, mais attention à utiliser dans des cas bien précis.

<img src="https://www.freepnglogos.com/uploads/twitter-logo-png/twitter-logo-vector-png-clipart-1.png" alt="" width="300"/>

<img src="https://tse1.mm.bing.net/th?id=OIP.EgSPriuEnAtlIWJV8R_E1QHaGs&pid=Api&rs=1&c=1&qlt=95&w=132&h=119" alt="" width="300"/>
<img src="https://tse3.mm.bing.net/th?id=OIP.XohozfrvQgQvlRBi16vj5QHaEK&pid=Api&P=0&w=301&h=170" alt="" width="500"/>

